In [5]:
import numpy as np
E=210000000
v=0.3
t=0.025
w=3000
xi,yi = (0,0)
xj,yj = (0.5,0.25)
xm,ym = (0,0.25)

xp,yp = (0,0)
xq,yq = (0.5,0)
xr,yr = (0.5,0.25)

A1 = (xi*(yj-ym)+xj*(ym-yi)+xm*(yi-yj))/2
A2 = (xp*(yq-yr)+xq*(yr-yp)+xr*(yp-yq))/2
B1 = (np.array([yj-ym,0,ym-yi,0,yi-yj,0,0,xm-xj,0,xi-xm,0,xj-xi,xm-xj,yj-ym,xi-xm,ym-yi,xj-xi,yi-yj]).reshape((3,6)))/(2*A1)
B2 = (np.array([yq-yr,0,yr-yp,0,yp-yq,0,0,xr-xq,0,xp-xr,0,xq-xp,xr-xq,yq-yr,xp-xr,yr-yp,xq-xp,yp-yq]).reshape((3,6)))/(2*A2)
D = (E/(1-v**2)) * (np.array([1,v,0,v,1,0,0,0,(1-v)/2]).reshape((3,3)))
B1_T = B1.transpose()
k1 = t*A1*B1_T
k11=np.matmul(k1,D)
k111=np.matmul(k11,B1)
#print(k111)
B2_T = B2.transpose()
k2 = t*A2*B2_T
k22=np.matmul(k2,D)
k222=np.matmul(k22,B2)
#print(k222)
tsm = np.zeros((8,8))

tsm[0:2,0:2] += k111[0:2,0:2]
tsm[0:2,4:8] += k111[0:2,2:6]
tsm[4:8,0:2] += k111[2:6,0:2]
tsm[4:8,4:8] += k111[2:6,2:6]

tsm[0:4,0:4] += k222[0:4,0:4]
tsm[0:4,4:6] += k222[0:4,4:6]
tsm[4:6,0:4] += k222[4:6,0:4]
tsm[4:6,4:6] += k222[4:6,4:6]
print ('Total Stiffness Matrix : \n' , tsm)
u=[0,0,1,1,1,1,0,0]
F = np.array([9.375,0,9.375,0]).reshape((4,1))


count = 0
for i in u:
  if i==1:
    count+=1

reduced_m = []
for i in range(8):
  for j in range(8):
    if u[i]*u[j] == 1:
      reduced_m.append(tsm[i][j])
reduced_m = np.array(reduced_m).reshape((count,count))
#print(reduced_m)

disp = np.linalg.solve(reduced_m, F)
print('Displacement: \n', disp)

disp_mat = np.zeros((8,1))
disp_mat[2:6,0] += disp[0:4,0]

reactions=np.matmul(tsm, disp_mat)
print("Reactions:\n",reactions)
u1 = np.zeros((6,1))
u1[2:4,0] += disp[2:4,0]
u2 = np.zeros((6,1))
u2[2:6,0] += disp[0:4,0]
sigma1 = np.matmul(np.matmul(D,B1),u1)
print('Sigma 1 : \n', sigma1)
sigma2 = np.matmul(np.matmul(D,B2),u2)
print('Sigma 2 : \n', sigma2)

Total Stiffness Matrix : 
 [[ 3461538.46153846        0.         -1442307.69230769   865384.61538462
         0.         -1875000.         -2019230.76923077  1009615.38461538]
 [       0.          6274038.46153846  1009615.38461538  -504807.69230769
  -1875000.                0.           865384.61538462 -5769230.76923077]
 [-1442307.69230769  1009615.38461538  3461538.46153846 -1875000.
  -2019230.76923077   865384.61538462        0.                0.        ]
 [  865384.61538462  -504807.69230769 -1875000.          6274038.46153846
   1009615.38461538 -5769230.76923077        0.                0.        ]
 [       0.         -1875000.         -2019230.76923077  1009615.38461538
   3461538.46153846        0.         -1442307.69230769   865384.61538462]
 [-1875000.                0.           865384.61538462 -5769230.76923077
         0.          6274038.46153846  1009615.38461538  -504807.69230769]
 [-2019230.76923077   865384.61538462        0.                0.
  -1442307.69230769  